In [34]:
import tensorflow as tf
import cv2
from cvzone.HandTrackingModule import HandDetector
# from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import tensorflow.keras

In [35]:
class Classifier:

    def __init__(self, modelPath, labelsPath=None):
        self.model_path = modelPath
        # Disable scientific notation for clarity
        np.set_printoptions(suppress=True)
        # Load the model
        self.model = tensorflow.keras.models.load_model(self.model_path)

        # Create the array of the right shape to feed into the keras model
        # The 'length' or number of images you can put into the array is
        # determined by the first position in the shape tuple, in this case 1.
        self.data = np.ndarray(shape=(1, 256, 256, 3), dtype=np.float32)
        self.labels_path = labelsPath
        if self.labels_path:
            label_file = open(self.labels_path, "r")
            self.list_labels = []
            for line in label_file:
                stripped_line = line.strip()
                self.list_labels.append(stripped_line)
            label_file.close()
        else:
            print("No Labels Found")

    def getPrediction(self, img, draw= True, pos=(50, 50), scale=2, color = (0,255,0)):
        # resize the image to a 224x224 with the same strategy as in TM2:
#         imgS = cv2.resize(img, (256, 256))
        # turn the image into a numpy array
        image_array = np.asarray(img)
        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

        # Load the image into the array
        self.data[0] = img

        # run the inference
        prediction = self.model.predict(self.data)
        indexVal = np.argmax(prediction)
        print(prediction, indexVal)

        if draw and self.labels_path:
            cv2.putText(img, str(self.list_labels[indexVal]),
                        pos, cv2.FONT_HERSHEY_COMPLEX, scale, color, 2)

        return list(prediction[0]), indexVal

In [ ]:
cap = cv2.VideoCapture(1)
detector = HandDetector(maxHands=1)

classifier = Classifier("model/imageclassifier.h5", "model/labels3.txt")
# model = tensorflow.keras.models.load_model("model/imageclassifier.h5")

# list_labels = []
# labels_path = labelsPath
# if labels_path:
#     label_file = open(labels_path, "r")
#     for line in label_file:
#         stripped_line = line.strip()
#         list_labels.append(stripped_line)
#     label_file.close()
# else:
#     print("No Labels Found")

offset = 20
imgSize = 256

folder = "letras"
counter = 0

labels = ["A", "B", "C", "D", "E", "F", "G", "I", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "Y"]

while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

        imgCropShape = imgCrop.shape

        aspectRatio = h / w

        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.shape
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
            resize = tf.image.resize(imgWhite, (256,256))
#             image = np.array(imgWhite)
            prediction, index = classifier.getPrediction(resize, draw=False)
#             print(prediction, index)

        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
            resize = tf.image.resize(imgWhite, (256,256))
#             image = np.array(imgWhite)
            prediction, index = classifier.getPrediction(resize, draw=False)

        cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                      (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
        cv2.rectangle(imgOutput, (x-offset, y-offset),
                      (x + w+offset, y + h+offset), (255, 0, 255), 4)


        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)

    cv2.imshow("Image", imgOutput)
    cv2.waitKey(1)

1/1 [==============================] - 0s 49ms/step
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.01020951 0.00000115 0.9897893  0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]] 10
1/1 [==============================] - 0s 14ms/step
[[0.         0.         0.         0.00071782 0.         0.00000002
  0.         0.00000058 0.         0.00140227 0.96310204 0.
  0.         0.         0.         0.         0.00003778 0.00000066
  0.03434581 0.00039298 0.        ]] 10
1/1 [==============================] - 0s 13ms/step
[[0.         0.         0.         0.00192182 0.         0.
  0.         0.         0.         0.         0.0000055  0.
  0.         0.00000056 0.         0.         0.         0.05943273
  0.9386394  0.         0.        ]] 18
1/1 [==============================] - 0s 14ms/step
[[0.         0.         0.         0.00000296 0.         0.00000002
  0.         0.         0.         0.000

1/1 [==============================] - 0s 13ms/step
[[0.         0.         0.00014116 0.         0.         0.
  0.         0.         0.00000019 0.         0.         0.
  0.9998586  0.         0.         0.         0.         0.
  0.         0.         0.        ]] 12
1/1 [==============================] - 0s 13ms/step
[[0.         0.         0.999992   0.         0.         0.
  0.         0.         0.00000019 0.         0.         0.
  0.00000782 0.         0.         0.         0.         0.
  0.         0.         0.        ]] 2
1/1 [==============================] - 0s 14ms/step
[[0.         0.         0.99611783 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.00388216 0.         0.         0.         0.         0.
  0.         0.         0.        ]] 2
1/1 [==============================] - 0s 14ms/step
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 2
1/1 [==============================] - 0s 13ms/step
[[0.         

1/1 [==============================] - 0s 14ms/step
[[0.         0.00299884 0.         0.         0.00000004 0.
  0.         0.00000001 0.00000407 0.00000167 0.0446105  0.
  0.         0.         0.00007649 0.         0.00200121 0.02623351
  0.9193606  0.00471305 0.        ]] 18
1/1 [==============================] - 0s 13ms/step
[[0.         0.0000001  0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.00001643 0.00003067
  0.9999528  0.         0.        ]] 18
1/1 [==============================] - 0s 13ms/step
[[0.         0.03205121 0.         0.         0.         0.00000006
  0.         0.         0.07025199 0.00008933 0.89541477 0.
  0.         0.         0.00025227 0.         0.00156639 0.00006002
  0.00015144 0.00016256 0.        ]] 10
1/1 [==============================] - 0s 13ms/step
[[0.         0.02341189 0.         0.         0.         0.
  0.         0.         0.00006857 0.000